In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#https://scipython.com/blog/quadtrees-2-implementation-in-python/
#https://pydoc.net/openslide-python/1.1.1/openslide/

import numpy as np
import openslide
from probreg import cpd
from probreg import transformation as tf
import cv2
from PIL import Image
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json

import pickle

In [3]:
# !pip install  -i https://test.pypi.org/simple/ qt-wsi-registration==0.0.3
import qt_wsi_reg.registration_tree as registration

In [4]:
source_slide_paths = list(Path(r"D:/Datasets/ColonCancer/MultiSlide/IHC").glob("*.tif"))
target_slide_paths  = list(Path(r"D:/Datasets/ColonCancer/MultiSlide/HE").glob("*.tif"))

if len(source_slide_paths) == 0:
    source_slide_paths = list(Path(r"/data/ScannerStudy/MultiSlide/IHC").glob("*.tif"))

if len(target_slide_paths) == 0:
    target_slide_paths = list(Path(r"/data/ScannerStudy/MultiSlide/HE").glob("*.tif"))

In [5]:
source_slide_dict = {name.stem.replace(" HE", ""): 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in source_slide_paths}
source_slide_dict.keys()

dict_keys(['CRC-A1-10', 'CRC-A1-5', 'CRC-A1-1', 'CRC-A1-13', 'CRC-A1-2'])

In [6]:
target_slide_dict = {name.stem.replace(" HE", ""):  
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in target_slide_paths}
target_slide_dict.keys()

dict_keys(['CRC-A1-10', 'CRC-A1-5', 'CRC-A1-1', 'CRC-A1-13', 'CRC-A1-2'])

In [7]:
for name in source_slide_dict:
    if name not in target_slide_dict:
        print(name)

In [8]:
for name in target_slide_dict:
    if name not in target_slide_dict:
        print(name)

In [ ]:
def add_help_fields(frame):
    
    frame["image_name_stem"] = [Path(image_name).stem for image_name in frame["image_name"]]
    
    frame["patient_id"] = [name.split("_")[2] for name in frame["image_name"]]

    frame["x1"] = [json.loads(vector.replace("\'","\""))['x1'] for vector in frame["vector"]]
    frame["y1"] = [json.loads(vector.replace("\'","\""))['y1'] for vector in frame["vector"]]

    frame["x2"] = [json.loads(vector.replace("\'","\""))['x2'] for vector in frame["vector"]]
    frame["y2"] = [json.loads(vector.replace("\'","\""))['y2'] for vector in frame["vector"]]

    frame["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    frame["center"] = [np.array((center_x, center_y)) for center_x, center_y in zip(frame["center_x"], frame["center_y"])]

    frame["anno_width"] = [x2-x1 for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["anno_height"]= [y2-y1 for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    return frame

landmarks = add_help_fields(pd.read_csv("../Validation/HE_IHC/GT_D240.csv"))
landmarks.head()

In [13]:
%%time

results = {}
#target_depth = 2

for thumbnail_size in [(1024, 1024)]: #(1024, 1024), (2048, 2048), (4096, 4096)
    
    for target_depth in [2]: # 0, 2
        
        result_folder = Path(f"Depth_{target_depth}/{thumbnail_size[0]}")

        result_folder.mkdir(parents=True, exist_ok=True)
    
        for name, source in source_slide_paths.items():

            parameter = {
                # feature extractor parameters
                "point_extractor": "sift",  #orb , sift
                "maxFeatures": 2048, 
                "crossCheck": False, 
                "flann": False,
                "ratio": 0.7, 
                "use_gray": False,

                # QTree parameter 
                "homography": True,
                "filter_outliner": False,
                "debug": True,
                "target_depth": target_depth,
                "run_async": True,
                "thumbnail_size": thumbnail_size
            }

            target = target_slide_paths[name]


            source_slide = source["slide"]
            target_slide = target["slide"]
            
            soure_path = source["name"]
            targert_path = target["name"]

            source_dimension = registration.Rect.create(registration.Rect, 0, 0, source_slide.dimensions[0], source_slide.dimensions[1])
            target_dimension = registration.Rect.create(registration.Rect, 0, 0, target_slide.dimensions[0], target_slide.dimensions[1])

            qtree = registration.RegistrationQuadTree(source_slide_path=soure_path, target_slide_path=targert_path, **parameter)

            results[source["name"]] = qtree
            
            #with open(result_folder / f'{qtree.source_name}-To-{qtree.target_name}.pickle', 'wb') as handle:
            #    pickle.dump(qtree, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
for path, qtree in results.items():
    
    #if qtree.target_depth != 2:
    #    continue
    
    df_gt = landmarks[landmarks["image_name"] == path.name]#.sample(n=5, random_state=1)
    
    annos = np.array(df_gt[["center_x", "center_y", "anno_width", "anno_height"]]) 
    result, _ = qtree.draw_annotations(annos, num_sub_pic=5, figsize=(10, 10))
    
    Path("RegResults").mkdir(exists_ok=True)
    result.savefig(f'RegResults/{qtree.source_slide_path.stem}_{qtree.target_slide_path.stem}.pdf')

In [17]:
for path, qtree in results.items():
        
    print(qtree)

Source: CRC-A1-1 
Target: CRC-A1-1 HE 
Source: (-0.50, 0.00, 149098.50, 146108.00) Target: (-0.50, -0.50, 140202.50, 146076.50)
x: [1.065, 0.006, -16708.524], y: [-0.006, 1.003, 2829.886]] error: 6615.017
nw: Source: (-0.50, 0.00, 74549.00, 73054.00) Target: (-0.26, 2828.98, 63076.26, 75649.79)
  x: [1.096, -0.011, -16423.440], y: [0.036, 1.027, -3059.340]] error: 5573.612
ne: Source: (74549.00, 0.00, 149098.50, 73054.00) Target: (63076.29, 2828.98, 140202.76, 75649.79)
  x: [1.013, -0.013, -525.635], y: [-0.039, 1.007, -793.200]] error: 5388.680
se: Source: (74549.00, 73054.00, 149098.50, 146108.00) Target: (63076.29, 75649.78, 140202.76, 146076.59)
  x: [1.014, 0.037, -1289.913], y: [-0.038, 1.003, -1402.082]] error: 3227.311
sw: Source: (-0.50, 73054.00, 74549.00, 146108.00) Target: (-0.26, 75649.78, 63076.26, 146076.59)
  x: [1.249, 0.092, -23922.243], y: [0.009, 1.154, -2525.129]] error: 17891.736
Source: CRC-A1-10 
Target: CRC-A1-10 HE 
Source: (-0.50, -0.50, 158126.50, 151834.50